In [1]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

df1 = pd.read_csv('/Volumes/IMVDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['sign', 'glyph'])

start_file = '/Volumes/IMVDrive/cfdb-django/media/start_file.csv'

if 'start_file.csv' in os.listdir('/Volumes/IMVDrive/cfdb-django/media/'):
    os.remove(start_file)

f=open(start_file, 'a+')

df_group = df1.groupby(by=['sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

print("Thresholding number of glyphs per sign at 50...")
first = True
for name,group in df_group:
    if first == True:
        group.reset_index().to_csv(f, header=True)
        first = False;
    elif first == False:
        group.reset_index().to_csv(f, header=None)
print("Thresholding finished.")

/anaconda/envs/labasi/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Thresholding number of glyphs per sign at 50...
Thresholding finished.


In [20]:
from random import shuffle
import glob
import sys
import numpy as np
from PIL import Image
from pathlib import Path

## ------------------------------------------------------------
## FORMAT CUTOM IMAGE DATA SET
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------------------------------------
## I use PIL instead of cv2
## ------------------------------------------------------------

print("FORMAT CUSTOM IMAGE INPUT")

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

#def _bytes_feature(value):
#    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def createDataRecord(outFileName, addrs, labels):
    print("")
    print('Creating '+outFileName+'...')
    writer = tf.python_io.TFRecordWriter(outFileName)
    for i in range(len(addrs)):
        if not i % 1000:
            print(outFileName+' data: {}/{}'.format(i,len(addrs)))
            sys.stdout.flush()
            
        filename = os.fsdecode('/Volumes/imvDrive/cfdb-django/media/glyph_img/'+addrs.iloc[i])
        if Path(filename).is_file():
            try:
                img = Image.open(filename).convert('L') #convert to grayscale
                img = img.resize((331,331)) 
                label = labels.iloc[i]
                ascii_label = int(''.join(str(ord(c)) for c in label))
                #print("label: "+str(label) + " / "+"ascii: "+str(ascii_label))

                feature_img = {
                    'glyph_img_raw': _bytes_feature(tf.compat.as_bytes(img.tostring())),#_bytes_feature(img.tobytes()), # 'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring()))
                    'label_raw': _int64_feature(ascii_label)
                }

                example = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example.SerializeToString()) 
                
            except Exception:
                pass
    writer.close()
    sys.stdout.flush()

FORMAT CUSTOM IMAGE INPUT


In [15]:
glyphtosignpath = pd.read_csv('/Volumes/IMVDRIVE/cfdb-django/media/start_file.csv')
addrs = glyphtosignpath.iloc[:,3]
labels = glyphtosignpath.iloc[:,2]

# divide the data into 80% train, 10% validation, and 10% test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
val_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]

In [21]:
createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)
print("DONE")


Creating train.tfrecords...
train.tfrecords data: 0/3808
train.tfrecords data: 1000/3808
train.tfrecords data: 2000/3808
train.tfrecords data: 3000/3808

Creating val.tfrecords...
val.tfrecords data: 0/476

Creating test.tfrecords...
test.tfrecords data: 0/477
DONE


In [5]:
'''
## ------------------------------
## READ INPUT: version 3
## --source: 
## https://damienpontifex.com/2017/12/02/inspecting-tfrecord-files-and-debugging-tensorflow-data-input/
## ------------------------------

def tfrec_data_input_fn(filenames, num_epochs=1, batch_size=64, shuffle=False):
    def _input_fn():
        def _parse_record(tf_record):
            features = {
                'image': tf.FixedLenFeature([], dtype=tf.string),
                'label': tf.FixedLenFeature([], dtype=tf.int64)
            }
            record = tf.parse_single_example(tf_record, features)

            image_raw = tf.decode_raw(record['image'], tf.float32)
            image_raw = tf.reshape(image_raw, shape=(331, 331, 1))

            label = tf.cast(record['label'], tf.int32)
            label = tf.one_hot(label, depth=2)

            return { 'image': image_raw }, label
        
        # For TF dataset blog post, see https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html
        dataset = tf.data.TFRecordDataset(filenames)
        dataset = dataset.map(_parse_record)

        if shuffle:
            dataset = dataset.shuffle(buffer_size=256)

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)

        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()

        return features, labels
    return _input_fn

tfrec_dev_input_fn = tfrec_data_input_fn(["/Volumes/IMVDRIVE/cfdb-django/train.tfrecords"], batch_size=3)
features, labels = tfrec_dev_input_fn()

with tf.Session() as sess:
    img, label = sess.run([features['image'], labels])
    print(img.shape, label.shape)

    # Loop over each example in batch
    for i in range(img.shape[0]):
        plt.imshow(img[i])
        plt.show()
        print('Class label ' + str(np.argmax(label[i])))
'''

'\n## ------------------------------\n## READ INPUT: version 3\n## --source: \n## https://damienpontifex.com/2017/12/02/inspecting-tfrecord-files-and-debugging-tensorflow-data-input/\n## ------------------------------\n\ndef tfrec_data_input_fn(filenames, num_epochs=1, batch_size=64, shuffle=False):\n    def _input_fn():\n        def _parse_record(tf_record):\n            features = {\n                \'image\': tf.FixedLenFeature([], dtype=tf.string),\n                \'label\': tf.FixedLenFeature([], dtype=tf.int64)\n            }\n            record = tf.parse_single_example(tf_record, features)\n\n            image_raw = tf.decode_raw(record[\'image\'], tf.float32)\n            image_raw = tf.reshape(image_raw, shape=(331, 331, 1))\n\n            label = tf.cast(record[\'label\'], tf.int32)\n            label = tf.one_hot(label, depth=2)\n\n            return { \'image\': image_raw }, label\n        \n        # For TF dataset blog post, see https://developers.googleblog.com/2017/09

In [31]:
## ------------------------------
## READ INPUT: version 1
## --source: 
## Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]',
## https://www.youtube.com/watch?v=bqeUmLCgsVw
## ------------------------------


import tensorflow as tf
import sys
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

channels = 1
batch_size = 1

sess = tf.Session()
sess.run(tf.global_variables_initializer())

def parser(record):
    keys_to_features = {
        "glyph_img_raw": tf.FixedLenFeature([], tf.string),
        "label_raw": tf.FixedLenFeature([], tf.int64)
    }
    
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["glyph_img_raw"], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, shape=[331, 331, channels])
    label = tf.cast(parsed["label_raw"], tf.int32)
    
    return image, label

def input_fn(filenames, train, batch_size=1, buffer_size=2048):
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    dataset = dataset.map(parser)
    
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
        num_repeat = None
    else:
        num_repeat = 1
        
    dataset = dataset.repeat(num_repeat)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    images_batch, labels_batch = iterator.get_next()
    x = {'image': images_batch}
    y = labels_batch

    return x, y

'''

def input_fn(filenames):
    feature = {'glyph_img_raw': tf.FixedLenFeature([], tf.string),
                'label_raw': tf.FixedLenFeature([], tf.int64)}
    
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
    
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)

    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['glyph_img_raw'], tf.float32)
    
    # Cast label data into int32
    label = tf.cast(features['label_raw'], tf.int32)
    print(features['label_raw'])

    # Reshape image data into the original shape
    image = tf.reshape(image, [331, 331, 1])
    
    return image, label

'''

def train_input_fn():
    return input_fn(filenames=['/Volumes/IMVDRIVE/cfdb-django/train.tfrecords'], train=True)

def val_input_fn():
    return input_fn(filenames=["/Volumes/IMVDRIVE/cfdb-django/val.tfrecords"], train=True)

def test_input_fn():
    return input_fn(filenames=["/Volumes/IMVDRIVE/cfdb-django/test.tfrecords"], train=False)

sess.run(init_op)

features, labels = train_input_fn()
img, label = sess.run([features['image'], labels])
print(img.shape, label.shape)

for i in range(img.shape[0]):
    print(img[i])
    #cv2.imshow('image', img[i])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    print('Class label'+str(np.argmax(label[i])))


for i in range(5):
    print("")
    print("                     iteration: "+str(i))
    image_batch, label_batch = train_input_fn()
    print("")
    print(image_batch)
    print(label_batch)
        #print("image_batch"+str(image_batch.shape)+" \ "+"label_batch"+str(label_batch.shape))


OutOfRangeError: Attempted to repeat an empty dataset infinitely.
	 [[Node: IteratorGetNext_23 = IteratorGetNext[output_shapes=[[?,331,331,1], [?]], output_types=[DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_23)]]

Caused by op 'IteratorGetNext_23', defined at:
  File "/anaconda/envs/labasi/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/labasi/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/labasi/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/labasi/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/envs/labasi/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda/envs/labasi/lib/python3.6/site-packages/zmq/

In [9]:
## ------------------------------
## VARIABLES
## ------------------------------
upload = pd.read_csv('/Volumes/imvDrive/cfdb-django/media/start_file.csv', usecols=['glyph', 'sign']) #
upload = upload.groupby(['sign']).nunique() #
sign_count = len(upload)-1 #
x = tf.placeholder(tf.float32, shape=[batch_size, 331, 331, 1])
labels = tf.placeholder(tf.float32, shape=[None, sign_count])
# y_true_cls = tf.argmax(labels, dimension=1)

## ------------------------------
## CONVOLUTIONAL NEURAL NETWORK
## Adapted from:
## https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
## https://www.tensorflow.org/tutorials/images/deep_cnn
## ------------------------------
# Count number of signs

def conv_layer1_fn(x):
    W1 = tf.Variable(tf.zeros([10,10,1,1], dtype=tf.float32))     
    conv1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding="SAME") #stides=[batch_stride x_stride y_stride depth_stride]
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    norm1 = tf.nn.local_response_normalization(pool1,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    return norm1
    
def conv_layer2_fn(layer):
    W2 = tf.Variable(tf.truncated_normal([5,5,channels,batch_size], stddev=0.05)) #https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
    conv2 = tf.nn.conv2d(layer, W2, strides=[1, 1, 1, 1], padding="SAME")
    norm2 = tf.nn.local_response_normalization(conv2,depth_radius=5,bias=1,alpha=1,beta=0.5,name=None)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
    return pool2

def flat_layer(layer):                  
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    return layer 

def fc_relu(layer):
    W3 = tf.Variable(tf.truncated_normal([6889], stddev=0.05))
    flat_W3 = flat_layer(W3)
    B = tf.Variable(tf.constant(0.03,shape=[int(layer.shape[1])]))
    std_hypothesis = tf.matmul(layer, flat_W3) + B
    fc_relu = tf.nn.relu(std_hypothesis)
    return fc_relu

conv_layer1 = conv_layer1_fn(x)
conv_layer2 = conv_layer2_fn(conv_layer1)
flat = flat_layer(conv_layer2)
fc_relu1 = fc_relu(flat)
fc_relu2 = fc_relu(fc_relu1)
    
## ------------------------------
## OPTIMIZATION SESSION
## Adapted from:
## https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/
## ------------------------------
    
y_pred = tf.nn.softmax(fc_relu2)
y_pred_cls = tf.argmax(y_pred, dimension=1, axis=None)
    
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc_relu2,labels=labels)
cost = tf.reduce_mean(cross_entropy)
    
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

## ------------------------------
## CATALYZING PROCESS
## ------------------------------

sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# Run training batches
x_train_batch, label_train_batch = train_input_fn() #data.train.next_batch(batch_size)
feed_dict_tr = {x: x_train_batch, labels: label_train_batch}
sess.run(optimizer, feed_dict=feed_dict_tr)
print("training batches.")

# Process validation batches
x_valid_batch, label_valid_batch = val_input_fn() #data.valid.next_batch(train_batch_size)
feed_dict_val = {x: x_valid_batch, labels: label_train_batch}
val_loss = sess.run(cost, feed_dict=feed_dict_val)
print("validation batches")    

TypeError: float() argument must be a string or a number, not 'dict'

In [ ]:
## ------------------------------
## PREDICTION
## ------------------------------
y_pred = tf.nn.softmax(fc_relu2)
y_pred_cls = tf.argmax(y_pred, dimension=1)
    